In [1]:
from manim import *
#from classes.christmas_fourier_classes import *
from PIL import Image
import jupyter_capture_output

video_scene = f" -v WARNING --disable_caching standing_wave_HoG_NDW_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s standing_wave_HoG_NDW_Scene"

Jupyter Capture Output v0.0.11


In [8]:
n = 5
TUBE_LENGTH = 10


# light mode colors
# MAIN_COLOR = BLACK
# red_color = PURE_RED
# blue_color = PURE_BLUE
# grey_color = DARK_GREY

# dark mode colors
MAIN_COLOR = WHITE
red_color = RED
blue_color = BLUE
grey_color = GREY


# returns incoming wave with x-dependence for given time
def get_wave_in(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    v_ph = lambda_wave * omega_wave / 2 / PI
    def wave_in(x):
        return -np.sin(k_wave*x - omega_wave*t)
    return wave_in


# returns x-dependent reflected wave for given time
def get_wave_out(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    v_ph = lambda_wave * omega_wave / 2 / PI
    def wave_out(x):
        return -np.sin(-k_wave*x - omega_wave*t)
    return wave_out


# superposition of the above two waves
def get_super_wave(t, wave_params):
    lambda_wave = wave_params[0]
    omega_wave = wave_params[1]
    k_wave = 2*PI / lambda_wave
    v_ph = lambda_wave * omega_wave / 2 / PI
    T = TUBE_LENGTH / v_ph
    if (t < T):
        def super_wave(x):
            if (x < v_ph*t):
                return get_wave_in(t, wave_params)(x)
            else:
                return 0
        return super_wave
    elif (t < 2*T):
        def super_wave(x):
            if (x < TUBE_LENGTH - v_ph*(t-T)):
                return get_wave_in(t, wave_params)(x)
            else:
                return get_wave_in(t, wave_params)(x) + get_wave_out(t, wave_params)(x)
        return super_wave
    else:
        def super_wave(x):
            return get_wave_in(t, wave_params)(x) + get_wave_out(t, wave_params)(x)
        return super_wave


# class creating a box to display incoming / outgoing transversal wave as well as their superposition
class StandingWaveTrans(Mobject):
    def __init__(self, position, height, width, **kwargs):
        super().__init__(**kwargs)
        line_top = Line(start = position + height/2*UP - width/2*RIGHT, end = position + height/2*UP + width/2*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        line_right = Line(start = position + height/2*UP + width/2*RIGHT, end = position - height/2*UP + width/2*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        line_bottom = Line(start = position - height/2*UP + width/2*RIGHT, end = position - height/2*UP - width/2*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        line_left = DashedLine(start = position - height/2*UP - width/2*RIGHT, end = position + height/2*UP - width/2*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        rectangle_open = VGroup(line_top, line_right, line_bottom, line_left)
        self.add(rectangle_open)
        separator = Line(start = position - np.array([width/2, 0, 0]), end = position + np.array([width/2, 0, 0]), stroke_width = 1, color = MAIN_COLOR)
        self.add(rectangle_open, separator)
        text_super = Tex(r"Superposition der Transversalwellen", font_size = 24, color = grey_color).next_to(separator, 0.5*DOWN + LEFT).shift(height/2 * UP).shift(width * RIGHT)
        text_in = Tex(r"Einlaufende Welle", font_size = 24, color = red_color).next_to(separator, 0.5*DOWN + RIGHT).shift(width * LEFT)
        text_out = Tex(r"Reflektierte Welle", font_size = 24, color = blue_color).next_to(separator, 0.5*DOWN + LEFT).shift(width * RIGHT)
        self.add(text_super, text_in, text_out)
        # number plane for the superposition
        self.super_npla = NumberPlane(
            x_range = [0, TUBE_LENGTH, TUBE_LENGTH / (2*n - 1)], y_range = [-3.25, 3.25, 1], x_length = width, y_length = 9/10*height/2,
            x_axis_config = {"stroke_opacity": 0.215, "stroke_color": MAIN_COLOR}, y_axis_config = {"stroke_opacity": 0.125, "stroke_color": MAIN_COLOR}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(position + np.array([0, height/4, 0]))
        self.add(self.super_npla)
        # number plane for the individual waves
        self.individual_npla = NumberPlane(
            x_range = [0, TUBE_LENGTH, TUBE_LENGTH / (2*n - 1)], y_range = [-3.25, 3.25, 1], x_length = width, y_length = 9/10*height/2,
            x_axis_config = {"stroke_opacity": 0.215, "stroke_color": MAIN_COLOR}, y_axis_config = {"stroke_opacity": 0.125, "stroke_color": MAIN_COLOR}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(position - np.array([0, height/4, 0]))
        self.add(self.individual_npla)

    # method to place gods hand to create the incoming wave
    def get_hand_of_god(self, t, wave_params):
        position_at_zero = self.individual_npla.c2p(0, get_wave_in(t, wave_params)(0))
        hand_of_god = ImageMobject("../external_media/HandOfGod4_trans.png").scale(0.4).next_to(position_at_zero, LEFT*0.2)
        return hand_of_god

    # method to return incoming wave (only defined for x < v_phase*t)
    def get_incoming(self, t, wave_params):
        lambda_wave = wave_params[0]
        omega_wave = wave_params[1]
        v_ph = lambda_wave * omega_wave / 2 / PI
        return self.individual_npla.plot(get_wave_in(t, wave_params), x_range = [0, min(v_ph*t, TUBE_LENGTH)], stroke_width = 2, stroke_color = red_color)

    # method to get the reflected wave (only defined for when v_phase has catched up)
    def get_reflected(self, t, wave_params):
        lambda_wave = wave_params[0]
        omega_wave = wave_params[1]
        v_ph = lambda_wave * omega_wave / 2 / PI
        T = TUBE_LENGTH / v_ph
        if (t > T):
            return self.individual_npla.plot(get_wave_out(t, wave_params), x_range = [max(0, T*v_ph + TUBE_LENGTH - v_ph*t), TUBE_LENGTH], stroke_width = 2, stroke_color = blue_color)
        else:
            return Dot(fill_opacity = 0)

    # method that gets superpositioned wave defined for the entire tube length
    def get_superposition(self, t, wave_params):
        return self.super_npla.plot(get_super_wave(t, wave_params), x_range = [0, TUBE_LENGTH], stroke_width = 2, stroke_color = grey_color)
    

# tube
class StandingWaveAcustic(Mobject):
    def __init__(self, position, height, width, **kwargs):
        super().__init__(**kwargs)
        self.top_left = position + height/2*UP - width/2*RIGHT
        self.bottom_left = position - height/2*UP - width/2*RIGHT
        line_top = Line(start = self.top_left, end = position + height/2*UP + width/2*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        line_right = Line(start = position + height/2*UP + width/2*RIGHT, end = position - height/2*UP + width/2*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        line_bottom = Line(start = position - height/2*UP + width/2*RIGHT, end = self.bottom_left, stroke_width = 2, color = MAIN_COLOR)
        line_left = DashedLine(start = self.bottom_left, end = self.top_left, stroke_width = 2, color = MAIN_COLOR)
        rectangle_open = VGroup(line_top, line_right, line_bottom, line_left)
        self.add(rectangle_open)
        # number plane to access the tube
        self.tube_npla = NumberPlane(
            x_range = [0, TUBE_LENGTH, TUBE_LENGTH / (2*n - 1)], y_range = [-1, 1, 1], x_length = width, y_length = height,
            x_axis_config = {"stroke_opacity": 0.215, "stroke_color": MAIN_COLOR}, y_axis_config = {"stroke_opacity": 0.125, "stroke_color": MAIN_COLOR}, background_line_style = {"stroke_opacity": 0.125}
        ).move_to(position)

    # returns a piston to create pressure wave
    def get_piston(self, t, wave_params):
        # create piston frame
        piston_width = 1
        piston_top_width = 0.2
        line_top = Line(start = self.top_left, end = self.top_left - piston_width*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        line_left = Line(start = self.top_left - RIGHT, end = self.bottom_left - RIGHT, stroke_width = 2, color = MAIN_COLOR)
        line_bottom = Line(start = self.bottom_left, end = self.bottom_left - piston_width*RIGHT, stroke_width = 2, color = MAIN_COLOR)
        piston_frame = VGroup(line_top, line_left, line_bottom)
        # get amplitude for given t at x = 0
        amplitude_at_zero = get_wave_in(t, wave_params)(0)
        piston_shift = amplitude_at_zero / 4 - 0.25
        # create top of the piston
        piston_top = Polygon(
            self.top_left + piston_shift*RIGHT, 
            self.bottom_left + piston_shift*RIGHT, 
            self.bottom_left - piston_top_width*RIGHT + piston_shift*RIGHT, 
            self.top_left-piston_top_width*RIGHT + piston_shift*RIGHT, 
            stroke_width = 2, color = MAIN_COLOR, fill_opacity = 0.75
        )
        # create the bottom of the piston
        piston_bottom_width = 0.1
        piston_bottom_right_center = (self.top_left+self.bottom_left) / 2 + piston_shift*RIGHT
        piston_bottom_left_center = (self.top_left+self.bottom_left) / 2 - piston_width*RIGHT
        piston_bottom = Polygon(
            piston_bottom_right_center - piston_top_width*RIGHT + piston_bottom_width/2 * UP,
            piston_bottom_right_center - piston_top_width*RIGHT - piston_bottom_width/2 * UP,
            piston_bottom_left_center - piston_bottom_width/2 * UP,
            piston_bottom_left_center + piston_bottom_width/2 * UP,  
            stroke_width = 2, color = MAIN_COLOR, fill_opacity = 1      
        )
        piston = VGroup(piston_frame, piston_top, piston_bottom)
        return piston


    # fills the tube with pressure according to the superposition of the waves
    def get_pressure(self, t, wave_params):
        superpositioned_wave = get_super_wave(t, wave_params)
        x_array = np.linspace(0, TUBE_LENGTH, 500)
        pressure_array = np.array([superpositioned_wave(x) for x in x_array]) / 4 + 0.5
        pressure_group = VGroup()
        for i, x in enumerate(x_array):
            pressure_group.add(Line(start = self.tube_npla.c2p(x, -1, 0), end = self.tube_npla.c2p(x, 1, 0), color = MAIN_COLOR, stroke_opacity = pressure_array[i], stroke_width = 2))
        return pressure_group

In [9]:
%%manim -qh --fps 60 $video_scene


class standing_wave_HoG_NDW_Scene(Scene):
    def construct(self):
        #self.camera.background_color = WHITE
        self.camera.background_color = "#20455A"
        CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
        # self.add(CVC)


        # wave parameters
        LAMBDA_WAVE = (4 * TUBE_LENGTH) / (2*n - 1)
        OMEGA_WAVE = 4
        wave_params = (LAMBDA_WAVE, OMEGA_WAVE)
        
        
        # wave box
        wave_trans_box = StandingWaveTrans(position = np.array([0.5, -0.5, 0]), height = 4.5, width = 10)
        self.add(wave_trans_box)

        # adding Gods hand [worship]
        hand_of_god = wave_trans_box.get_hand_of_god(0, wave_params)
        hand_of_god.get_method = wave_trans_box.get_hand_of_god
        self.add(hand_of_god) 

        # adding the waves to the box
        incoming_wave = wave_trans_box.get_incoming(0, wave_params)
        incoming_wave.get_method = wave_trans_box.get_incoming
        self.add(incoming_wave)

        reflected_wave = wave_trans_box.get_reflected(0, wave_params)
        reflected_wave.get_method = wave_trans_box.get_reflected
        self.add(reflected_wave)

        superpositioned_wave = wave_trans_box.get_superposition(0, wave_params)
        superpositioned_wave.get_method = wave_trans_box.get_superposition
        self.add(superpositioned_wave)


        # gas tube
        gas_tube_box = StandingWaveAcustic(position = np.array([0.5, 2.75, 0]), height = 1.75/2, width = 10)
        self.add(gas_tube_box)

        # piston
        gas_piston = gas_tube_box.get_piston(0, wave_params)
        gas_piston.get_method = gas_tube_box.get_piston
        self.add(gas_piston)

        # add pressure to the gas tube
        gas_pressure = gas_tube_box.get_pressure(0, wave_params)
        gas_pressure.get_method = gas_tube_box.get_pressure
        self.add(gas_pressure)


        # updates the waves using the ValueTracker time 't'
        def wave_updater(wave):
            t = t_tracker.get_value()
            wave.become(wave.get_method(t, wave_params))


        self.wait(1)
        t_tracker = ValueTracker(0)
        incoming_wave.add_updater(wave_updater)
        reflected_wave.add_updater(wave_updater)
        superpositioned_wave.add_updater(wave_updater)
        hand_of_god.add_updater(wave_updater)
        gas_pressure.add_updater(wave_updater)
        gas_piston.add_updater(wave_updater)
        self.play(t_tracker.animate.set_value(20), rate_func = linear, run_time = 40)
        self.wait(5)

Manim Community v0.18.1